In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys

from pandas.io.json import json_normalize
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os
os.getcwd()

'/home/mrk_ram108/Kaggle'

In [2]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [str(column)+"."+str(subcolumn) for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print("Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
#train = load_df(os.getcwd()+'/train.csv')
#test = load_df(os.getcwd()+'/test.csv')

#### Saving CSV files

In [4]:
#train.to_csv('train_extracted.csv',encoding='utf8')
#test.to_csv('test_extracted.csv', encoding = 'utf8')

## Reading the data
train = pd.read_csv('train_extracted.csv',encoding='utf8',dtype={'fullVisitorId': 'str'})
test = pd.read_csv('test_extracted.csv', encoding = 'utf8',dtype={'fullVisitorId': 'str'})

/home/mrk_ram108/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Combining train and test data

In [5]:
train['data_source'] = 'train'
test['data_source'] = 'test'

all_data = pd.concat([train,test], axis = 0)

/home/mrk_ram108/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
all_data.shape

(1708337, 57)

#### Emtpy values 

In [7]:
100 * float(train['totals.transactionRevenue'].isnull().sum())/len(train['totals.transactionRevenue'])

98.72572768529513

#### Analysis


In [8]:
all_data.drop([u'Unnamed: 0'], axis = 1, inplace = True)
all_data.head()

,channelGrouping,data_source,date,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,...,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime
0,Organic Search,train,20160902,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,(not provided),organic,NaN,google,1472830385,1,1472830385
1,Organic Search,train,20160902,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,(not provided),organic,NaN,google,1472880147,1,1472880147
2,Organic Search,train,20160902,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,(not provided),organic,NaN,google,1472865386,1,1472865386
3,Organic Search,train,20160902,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,google + online,organic,NaN,google,1472881213,1,1472881213
4,Organic Search,train,20160902,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,...,(not set),NaN,True,(not provided),organic,NaN,google,1472822600,2,1472822600


In [9]:
all_data.visitNumber.describe()

count    1.708337e+06
mean     2.335170e+00
std      9.354034e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      4.570000e+02
Name: visitNumber, dtype: float64

#### Finding columns with just one value in it

In [10]:
### Selecting columns with just one unique value and those with NULL values greater than 50% of the dataset
drop_columns = [each for each in all_data.columns if len(train[each].unique()) <= 2 or all_data[each].isnull().sum() > 0.5 * all_data.shape[0]]
drop_columns = [each for each in drop_columns if each not in ['totals.transactionRevenue','data_source']]

#### Response variable

In [11]:
all_data['totals.transactionRevenue'] = pd.to_numeric(all_data['totals.transactionRevenue'])
all_data['totals.transactionRevenue'].describe()

count    1.151500e+04
mean     1.337448e+08
std      4.482852e+08
min      1.000000e+04
25%      2.493000e+07
50%      4.945000e+07
75%      1.076550e+08
max      2.312950e+10
Name: totals.transactionRevenue, dtype: float64

In [12]:
all_data_upd = all_data.drop(drop_columns, axis = 1)

In [13]:
for each in all_data_upd.columns:
    print(each)
    print(all_data_upd[each].value_counts())
    print("\n")

channelGrouping
Organic Search    738963
Social            354971
Direct            273134
Referral          211307
Display            51283
Paid Search        45627
Affiliates         32915
(Other)              137
Name: channelGrouping, dtype: int64


data_source
train    903653
test     804684
Name: data_source, dtype: int64


date
20171212    9234
20171213    9131
20171004    5122
20170920    4880
20161128    4807
20170921    4715
20161115    4685
20171005    4679
20161114    4466
20161130    4435
20180321    4434
20161026    4375
20161129    4337
20161116    4334
20161004    4322
20161205    4265
20171017    4258
20180327    4227
20170426    4224
20161201    4200
20171127    4194
20161027    4162
20161121    4143
20171108    4136
20161117    4074
20161024    4063
20161110    4055
20171006    4033
20161103    4014
20180226    3989
            ... 
20170430    1594
20161001    1589
20161226    1586
20170115    1576
20170429    1566
20170611    1555
20170204    1549
20170423    1548


United States               717217
India                       105317
United Kingdom               73341
Canada                       51057
Germany                      38516
Japan                        36637
Brazil                       35432
Vietnam                      34869
France                       32289
Thailand                     29859
Turkey                       29166
Taiwan                       26560
Mexico                       25270
Australia                    23660
Spain                        23431
Netherlands                  22146
Italy                        20694
Russia                       19641
Indonesia                    16675
Poland                       15874
Philippines                  14272
Singapore                    13294
Ireland                      12287
South Korea                  11563
Malaysia                     10820
Argentina                    10128
Sweden                        9895
Ukraine                       9859
Romania             

6578231451745091419_1512114073    2
128747466653784455_1479283175     2
22780653525416747_1511683082      2
1125646319189533585_1489219057    2
4357202989256557726_1500188130    2
3354399941800427193_1481355758    2
6187281368753177854_1471935527    2
697400110005038316_1472021848     2
2719373285682504427_1474009073    2
1692266033849940301_1520582070    2
8775495552152201078_1501657013    2
0982833231079207283_1496905133    2
8850375778604812895_1481874986    2
3514720393317202630_1514706758    2
1112657102058195092_1470380158    2
0049220146968431383_1472626229    2
5155411368235514331_1495349915    2
9328692348677910181_1490338007    2
1281779287366182489_1510991898    2
0405996727678488611_1500187709    2
4274511861314211339_1500015203    2
1555790315581935845_1475131695    2
2405822934418209148_1505458219    2
5997647927187253437_1510214247    2
7094476618461750872_1516867182    2
8907178468365394504_1524466316    2
7483600664917507409_1519545171    2
1748279244392362923_14809246

google                                               817892
youtube.com                                          329450
(direct)                                             273152
mall.googleplex.com                                  120541
analytics.google.com                                  37436
Partners                                              32931
sites.google.com                                       9072
google.com                                             8341
gdeals.googleplex.com                                  8298
m.facebook.com                                         6737
dfa                                                    6704
baidu                                                  4971
reddit.com                                             4092
facebook.com                                           3584
googleads.g.doubleclick.net                            3539
qiita.com                                              3394
bing                                    

#### Creating date variables

In [14]:
all_data_upd['year'] = all_data_upd.date.apply(lambda x: int(str(x)[0:4]))
all_data_upd['month'] = all_data_upd.date.apply(lambda x: int(str(x)[4:6]))
all_data_upd['day'] = all_data_upd.date.apply(lambda x: int(str(x)[6:8]))

#### Cleaning and imputing train and test separately - since we do not want to use the test data to impute the training data

In [15]:
train_upd = all_data_upd[all_data_upd.data_source == 'train']
test_upd = all_data_upd[all_data_upd.data_source == 'test']

print(train_upd.shape)
print(test_upd.shape)

(903653, 27)
(804684, 27)


#### Train

In [16]:
train_upd.head()

,channelGrouping,data_source,date,device.browser,device.deviceCategory,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,totals.transactionRevenue,trafficSource.campaign,trafficSource.medium,trafficSource.source,visitId,visitNumber,visitStartTime,year,month,day
0,Organic Search,train,20160902,Chrome,desktop,Windows,1131660440785968503,Izmir,Asia,Turkey,...,NaN,(not set),organic,google,1472830385,1,1472830385,2016,9,2
1,Organic Search,train,20160902,Firefox,desktop,Macintosh,377306020877927890,not available in demo dataset,Oceania,Australia,...,NaN,(not set),organic,google,1472880147,1,1472880147,2016,9,2
2,Organic Search,train,20160902,Chrome,desktop,Windows,3895546263509774583,Madrid,Europe,Spain,...,NaN,(not set),organic,google,1472865386,1,1472865386,2016,9,2
3,Organic Search,train,20160902,UC Browser,desktop,Linux,4763447161404445595,not available in demo dataset,Asia,Indonesia,...,NaN,(not set),organic,google,1472881213,1,1472881213,2016,9,2
4,Organic Search,train,20160902,Chrome,mobile,Android,27294437909732085,not available in demo dataset,Europe,United Kingdom,...,NaN,(not set),organic,google,1472822600,2,1472822600,2016,9,2


In [17]:
## Cleaning the character columns in the data

#Float columns
float_cols = []


for each in train_upd.columns:
    
    if(each == 'data_source'):
        continue
        
    if(train_upd[each].dtype == 'O' and len(train_upd[each].unique()) < 500000):
        print(each)
        ### Skipping the values are actually numeric or float
        
        if( not (len(pd.to_numeric(train_upd[each],errors = 'coerce').unique()) < 0.5 * len(train_upd[each].unique())) ):
            train_upd.loc[:,each] = train_upd[each].fillna(train_upd[each].mean())
            float_cols.append(each)
            
        else:


            print(len(train_upd[each].unique()))
            train_upd.loc[:,each] = train_upd[each].apply(lambda x: x.lower().strip())
            extreme_values = train_upd[each].value_counts()[train_upd[each].value_counts() < 10].index

            ## Replacing the outliers (extreme) values with 'Others'

            train_upd.loc[:,each] = train_upd[each].apply(lambda x: np.where(x in extreme_values, 'Others',x))

channelGrouping
8


/home/mrk_ram108/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


device.browser
54
device.deviceCategory
3
device.operatingSystem
20
geoNetwork.city
649
geoNetwork.continent
6
geoNetwork.country
222
geoNetwork.metro
94
geoNetwork.networkDomain
28064
geoNetwork.region
376
geoNetwork.subContinent
23
trafficSource.campaign
10
trafficSource.medium
7
trafficSource.source
380


#### Test

In [18]:
## Cleaning the character columns in the data

for each in test_upd.columns:
    
    
    if(each == 'data_source'):
        continue
    
    print(each)
    if(test_upd[each].dtype == 'O' and len(test_upd[each].unique()) < 500000):
            
        ### Skipping the values are actually numeric or float
        
        ### Identifying the columns as numeric/float as per the training data
        if(each in float_cols):
            
            train_upd[each] = train_upd[each].fillna(train_upd[each].mean())
        else:


            print(len(train_upd[each].unique()))
            train_upd[each] = train_upd[each].apply(lambda x: x.lower().strip())
            extreme_values = train_upd[each].value_counts()[train_upd[each].value_counts() < 10].index

            ## Replacing the outliers (extreme) values with 'Others'

            train_upd[each] = train_upd[each].apply(lambda x: np.where(x in extreme_values, 'Others',x))

channelGrouping
8


/home/mrk_ram108/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mrk_ram108/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


date
device.browser
27
device.deviceCategory
3
device.operatingSystem
15
fullVisitorId
geoNetwork.city
482
geoNetwork.continent
6
geoNetwork.country
188
geoNetwork.metro
79
geoNetwork.networkDomain
3017
geoNetwork.region
318
geoNetwork.subContinent
23
sessionId
totals.hits
totals.pageviews
totals.transactionRevenue
trafficSource.campaign
9
trafficSource.medium
7
trafficSource.source
124
visitId
visitNumber
visitStartTime
year
month
day


#### Combining the data again

In [19]:
all_data_upd = pd.concat([train_upd,test_upd], axis = 0)

In [20]:
## Removing 'geoNetwork.networkDomain' column for now, since it has a lot of categories - over 3000

all_data_upd = all_data_upd.drop(['device.browser','geoNetwork.metro','geoNetwork.networkDomain','geoNetwork.region','trafficSource.source','geoNetwork.city','geoNetwork.country'], axis = 1)
all_data_upd.shape

(1708337, 20)

#### Dividing the data into train and test

In [21]:
all_data_upd['totals.transactionRevenue'] = all_data_upd['totals.transactionRevenue'].fillna(0)
all_data_upd = all_data_upd.fillna(0)

In [22]:
### Removing id columns
id_cols = ['sessionId','fullVisitorId','visitId','date']
all_data_dummies = pd.get_dummies(all_data_upd.drop(id_cols, axis = 1), dummy_na = True)

In [23]:
all_data_dummies.shape

(1708337, 175)

#### Saving the transformed train data

In [24]:
# all_data_dummies.to_csv('all_data_dummies.csv', index = False, encoding = 'utf8')
#all_data_dummies = pd.read_csv("all_data_dummies.csv", encoding = 'utf8')

In [25]:
all_data_dummies.shape

(1708337, 175)

In [26]:
train_dummies = all_data_dummies[all_data_dummies.data_source_train == 1]
test_dummies = all_data_dummies[all_data_dummies.data_source_test == 1]

# train_dummies = all_data_upd[all_data_upd.data_source == 'train']
# test_dummies = all_data_upd[all_data_upd.data_source == 'test']

In [27]:
train_dummies = train_dummies.drop(['data_source_train','data_source_test','data_source_nan'], axis = 1)
test_dummies = test_dummies.drop(['data_source_train','data_source_test','data_source_nan','totals.transactionRevenue'], axis = 1)

#train_dummies = train_dummies.drop(['data_source'], axis = 1)
#test_dummies = test_dummies.drop(['data_source','totals.transactionRevenue'], axis = 1)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(train_dummies.drop(['totals.transactionRevenue'], axis = 1), \
                  train_dummies['totals.transactionRevenue'],test_size = 0.2, random_state = 9)

In [29]:
X_train.head()

,totals.hits,totals.pageviews,visitNumber,visitStartTime,year,month,day,channelGrouping_(Other),channelGrouping_(other),channelGrouping_Affiliates,...,trafficSource.campaign_test-liyuhz,trafficSource.campaign_nan,trafficSource.medium_(none),trafficSource.medium_(not set),trafficSource.medium_affiliate,trafficSource.medium_cpc,trafficSource.medium_cpm,trafficSource.medium_organic,trafficSource.medium_referral,trafficSource.medium_nan
40485,4,4.0,1,1498884588,2017,6,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
784765,1,1.0,1,1501200752,2017,7,27,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103499,1,1.0,1,1499169217,2017,7,4,0,0,0,...,0,0,0,0,0,0,0,1,0,0
363033,1,1.0,4,1482550584,2016,12,23,0,0,0,...,0,0,1,0,0,0,0,0,0,0
886064,3,2.0,5,1500310627,2017,7,17,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
print(X_train.shape)
print(y_train.shape)

(722922, 171)
(722922,)


In [31]:
lr_model = LinearRegression(n_jobs = -1)

In [32]:
lr_model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [33]:
predict = lr_model.predict(X_train)
predict_test = lr_model.predict(X_test)

In [34]:
np.histogram(predict)

(array([716501,   5739,    539,     79,     37,     13,      1,      8,
             2,      3]),
 array([-1.25905828e+07,  4.04889368e+07,  9.35684564e+07,  1.46647976e+08,
         1.99727496e+08,  2.52807015e+08,  3.05886535e+08,  3.58966054e+08,
         4.12045574e+08,  4.65125094e+08,  5.18204613e+08]))

In [58]:
lmse = np.sum(np.log((y_train.values - np.where(predict < 1.04889368e+09, 0, predict)) ** 2 + 1))/len(y_train.values)
lmse

0.45331784935534464

In [64]:
lmse = np.sum(np.log((y_test.values - np.where(predict_test < 1.04889368e+9, 0, predict_test)) ** 2 + 1))/len(y_test.values)
lmse

0.4579103000800032

#### Predicting the test class

In [71]:
predict_actual_test = lr_model.predict(test_dummies)
predict_actual_t = np.log(np.where(predict_actual_test < 1e+9, 0,predict_actual_test)+1)

In [72]:
submission = pd.concat([test.fullVisitorId,pd.DataFrame(predict_actual_t)], axis = 1)
submission.columns.values[1]= 'PredictedLogRevenue'

In [73]:
submission_out = submission.groupby('fullVisitorId',as_index = False).sum()

In [74]:
submission_out.to_csv("kaggle_submission.csv", index = False)

#### Random forest

In [75]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
rf_classifier = RandomForestClassifier()

In [ ]:
#### To do:

## For visualiation: https://www.kaggle.com/pavansanagapati/simple-exploration-lgbm-model-lb-1-4187 
## Implement the globe

## For categorical features - implement the Encoders and proper imputation
## For columns with webties - extract the major website name (google.com/google.in have the same major content google)
## Numerical features - better imputation and try to create more variables


## Futher, implement a logistic regression and then perfrom a non-linear model on top
